In [1]:
import transformers

from transformers import BertTokenizer, BertModel

/home/erland/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_model

config.json: 100%|██████████| 570/570 [00:00<00:00, 4.88MB/s]
model.safetensors: 100%|██████████| 440M/440M [01:10<00:00, 6.24MB/s] 


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [2]:
from datasets import load_dataset

dataset = load_dataset("glue", "mrpc")
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
dataset["train"]["label"]

[1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,


In [3]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [2]:
import torch
from transformers import BertModel, BertTokenizer

bert_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
model = BertModel.from_pretrained(bert_model_name, output_hidden_states=True)

input_text = "This is a sample sentence."
inputs = tokenizer(input_text, return_tensors="pt")

outputs = model(**inputs)

hidden_states = outputs.hidden_states

print(f"Shape of the last hidden state: {hidden_states[-1].shape}")

print("\nShapes of all hidden states:")
for i, hidden_state in enumerate(hidden_states):
    print(f"Layer {i}: {hidden_state.shape}")

Shape of the last hidden state: torch.Size([1, 8, 768])

Shapes of all hidden states:
Layer 0: torch.Size([1, 8, 768])
Layer 1: torch.Size([1, 8, 768])
Layer 2: torch.Size([1, 8, 768])
Layer 3: torch.Size([1, 8, 768])
Layer 4: torch.Size([1, 8, 768])
Layer 5: torch.Size([1, 8, 768])
Layer 6: torch.Size([1, 8, 768])
Layer 7: torch.Size([1, 8, 768])
Layer 8: torch.Size([1, 8, 768])
Layer 9: torch.Size([1, 8, 768])
Layer 10: torch.Size([1, 8, 768])
Layer 11: torch.Size([1, 8, 768])
Layer 12: torch.Size([1, 8, 768])


In [5]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer

bert_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
model = BertForSequenceClassification.from_pretrained(
    bert_model_name, num_labels=2
)  # Assuming binary classification

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
%cd ../

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa74b2ebcd0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fa88c77cd90, raw_cell="%cd ../" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X33sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

/home/erland/Documents/Python Project/berrrt
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa74b2ebcd0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fa88c77e170, execution_count=2 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fa88c77cd90, raw_cell="%cd ../" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X33sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [4]:
from berrrt.utils import compute_metrics

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa74b2ebcd0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fa88c77cd90, raw_cell="from berrrt.utils import compute_metrics" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X34sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa74b2ebcd0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fa88c77df60, execution_count=4 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fa88c77cd90, raw_cell="from berrrt.utils import compute_metrics" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X34sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [7]:
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset

# Load MRPC dataset
dataset = load_dataset("glue", "mrpc")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


# Tokenize the inputs (text pairs)
def tokenize_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )


tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
for param in model.base_model.parameters():
    param.requires_grad = False

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa74b2ebcd0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fa74b1cb160, raw_cell="from transformers import (
    BertTokenizer,
    .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X32sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 10/345 [00:02<01:27,  3.81it/s]

{'loss': 0.6659, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.09}


                                                
  3%|▎         | 10/345 [00:05<01:27,  3.81it/s]

...

100%|██████████| 345/345 [03:32<00:00,  1.62it/s]

{'train_runtime': 212.424, 'train_samples_per_second': 51.802, 'train_steps_per_second': 1.624, 'train_loss': 0.6373010524805042, 'epoch': 3.0}


TrainOutput(global_step=345, training_loss=0.6373010524805042, metrics={'train_runtime': 212.424, 'train_samples_per_second': 51.802, 'train_steps_per_second': 1.624, 'train_loss': 0.6373010524805042, 'epoch': 3.0})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa74b2ebcd0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fa74b1caf50, execution_count=7 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fa74b1cb160, raw_cell="from transformers import (
    BertTokenizer,
    .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X32sZmlsZQ%3D%3D> result=TrainOutput(global_step=345, training_loss=0.6373010524805042, metrics={'train_runtime': 212.424, 'train_samples_per_second': 51.802, 'train_steps_per_second': 1.624, 'train_loss': 0.6373010524805042, 'epoch': 3.0})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [8]:
trainer.evaluate()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa74b2ebcd0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fa74b195c00, raw_cell="trainer.evaluate()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X35sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

100%|██████████| 13/13 [00:03<00:00,  4.17it/s]


{'eval_loss': 0.6226333975791931,
 'eval_accuracy': 0.6838235294117647,
 'eval_f1': 0.8122270742358079,
 'eval_precision': 0.6838235294117647,
 'eval_recall': 1.0,
 'eval_runtime': 3.3891,
 'eval_samples_per_second': 120.384,
 'eval_steps_per_second': 3.836,
 'epoch': 3.0}

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa74b2ebcd0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fa74b197a00, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fa74b195c00, raw_cell="trainer.evaluate()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X35sZmlsZQ%3D%3D> result={'eval_loss': 0.6226333975791931, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_precision': 0.6838235294117647, 'eval_recall': 1.0, 'eval_runtime': 3.3891, 'eval_samples_per_second': 120.384, 'eval_steps_per_second': 3.836, 'epoch': 3.0}>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [6]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
from datasets import Dataset
import pandas as pd

# Example DataFrame
df = pd.DataFrame(
    {
        "text": ["This is a sample sentence.", "Another example sentence."],
        "label": [0, 1],
    }
)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split dataset into training and validation sets
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1/1 [00:00<00:00, 293.84 examples/s]


In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # Output directory for model checkpoints
    num_train_epochs=3,  # Total number of training epochs
    per_device_train_batch_size=8,  # Batch size per device during training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # Strength of weight decay
    logging_dir="./logs",  # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",  # Evaluate each `logging_steps`
    report_to=None,
)

In [7]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,  # The instantiated 🤗 Transformers model to be trained
    args=training_args,  # Training arguments, defined above
    train_dataset=tokenized_datasets["train"],  # Training dataset
    eval_dataset=tokenized_datasets["test"],  # Evaluation dataset
    compute_metrics=compute_metrics,  # Function for calculating metrics, if needed
)

trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erlandpg. Use `wandb login --relogin` to force relogin


                                             
 67%|██████▋   | 2/3 [00:00<00:00,  2.81it/s]

{'eval_loss': 0.3672877550125122, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 0.0525, 'eval_samples_per_second': 19.049, 'eval_steps_per_second': 19.049, 'epoch': 1.0}


                                             
100%|██████████| 3/3 [00:00<00:00,  3.46it/s]

{'eval_loss': 0.36771246790885925, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 0.0861, 'eval_samples_per_second': 11.621, 'eval_steps_per_second': 11.621, 'epoch': 2.0}


                                             
100%|██████████| 3/3 [00:01<00:00,  2.75it/s]

{'eval_loss': 0.36857107281684875, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 0.0868, 'eval_samples_per_second': 11.519, 'eval_steps_per_second': 11.519, 'epoch': 3.0}
{'train_runtime': 5.2465, 'train_samples_per_second': 0.572, 'train_steps_per_second': 0.572, 'train_loss': 1.1022586822509766, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=1.1022586822509766, metrics={'train_runtime': 5.2465, 'train_samples_per_second': 0.572, 'train_steps_per_second': 0.572, 'train_loss': 1.1022586822509766, 'epoch': 3.0})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f903d2858d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f905d3238b0, execution_count=10 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f905d320250, raw_cell="from transformers import Trainer

trainer = Traine.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/erland/Documents/Python%20Project/berrrt/notebooks/breakdown_bert.ipynb#X12sZmlsZQ%3D%3D> result=TrainOutput(global_step=3, training_loss=1.1022586822509766, metrics={'train_runtime': 5.2465, 'train_samples_per_second': 0.572, 'train_steps_per_second': 0.572, 'train_loss': 1.1022586822509766, 'epoch': 3.0})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [11]:
tokenized_datasets.set_format("torch")
tokenized_datasets["train"]["input_ids"]

tensor([[ 101, 2178, 2742, 6251, 1012,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [14]:
result = model(
    tokenized_datasets["train"]["input_ids"],
    tokenized_datasets["train"]["attention_mask"],
    tokenized_datasets["train"]["label"],
)

In [15]:
result.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [16]:
result.loss

AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'loss'

In [1]:
from berrrt.dataset import BERRRTDataset
from hydra import compose, initialize
from omegaconf import OmegaConf


with initialize(version_base=None, config_path="../berrrt/conf"):
    cfg = compose(config_name="config")

dataset = BERRRTDataset(cfg)
dataset

/home/erland/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 1725/1725 [00:01<00:00, 1078.93 examples/s]


Dataset({
    features: ['label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})

In [1]:
from berrrt.dataset import BERRRTDataset
from hydra import compose, initialize
from omegaconf import OmegaConf


with initialize(version_base=None, config_path="../berrrt/conf"):
    cfg = compose(config_name="config")

dataset = BERRRTDataset(cfg)
dataset

/home/erland/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 1725/1725 [00:00<00:00, 10076.61 examples/s]


In [2]:
dataset.train_encoded

Dataset({
    features: ['label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})

In [3]:
model

NameError: name 'model' is not defined